
## TASK: Load JSON data to SQL Database

In [3]:
# -*- coding:utf-8 -*-
import sys, json
import simplejson
import sqlite3
import pandas as pd

"""
Given a DB of entities and their associated likes in JSON, produce a SQL DB of it.
If john@google.com has the following "like": {"name":"The Tonight Show Starring Jimmy Fallon",
"category":"TV Show","id":"31732483895","created_time":"2016-03-26T23:07:58+0000"},
then a row in the database would exist with the following:

email,category,name,id,created_time
john@google.com,"TV Show","The Tonight Show Starring Jimmy Fallon","2016-03-26T23:07:58"

The database is created in males_likes.sqlite

Ref. API: https://docs.python.org/2/library/sqlite3.html
"""


def url2json(uri_with_json):
    response = urllib.request.urlopen(uri_with_json)
    a = str(response.read())  
    j = json.loads(a) # decode JSON format
    return j

def javascript2json(url):
    import requests
    return requests.get(url).json()  #return json.loads(js.decode("utf-8"))

def json2sql(json_path, db_file):# Format the data to extract our fields of interest
    entries = []
    try:
        # READ JSON
        for line in open(json_path, 'r'):
            entries.append(simplejson.loads(line)) # use simplejson.loads() for more accurate error explanation
        datalist = json.loads(json.dumps(entries))

        # CREATE SQL TABLE (no PRIMARY KEY) with table_cols = [u'email',u'category', u'name', u'id',u'created_time']
        conn = sqlite3.connect(db_file)
        c = conn.cursor()
        #c.execute('''DROP TABLE male_likes;''')
        c.execute('''CREATE TABLE if not exists male_likes (email text, category text, name text, id text, created_time text);''')
        conn.commit()

        # HOW THE DATA LOOKS LIKE:
        all_entity_ids = set()
        columns = list(datalist[0].keys())
        likes_cols = datalist[0]['likes'][0].keys()
        print "Columns: ",columns, "\n Likes columns: ", likes_cols

        # INSERT EACH LIKE IN A ROW FOR EACH NON NULL EMAIL (other features can be null)
        for entry in datalist:
            if 'email' in entry['user'].keys():
                email = entry['user']['email']
                for like in entry['likes']:
                    category = like['category'] if 'category' in like else None
                    name = like['name'] if 'name' in like else None
                    id_ = like['id'] if 'id' in like else None
                    created_time = like['created_time'] if 'created_time' in like else None
                    c.execute("INSERT INTO male_likes (email, category, name, id, created_time) VALUES (?,?,?,?,?)",(email, category, name, id_, created_time))
                
        # READ SQLITE FULL DB INTO A RETURNED PANDAS DataFrame
        likes = pd.read_sql_query("SELECT * from male_likes", conn)
        conn.commit()
        c.close()
        return likes
    
    except ValueError:
        print "Decoding failed: ", ValueError
        pass


json_path = './data/facebook_males.json'
db = "male_likes.sqlite"
likes = json2sql(json_path, db)
likes.head(5)


# ACCESSING THE CREATED SQL DB:
conn = sqlite3.connect(db)
c = conn.cursor()
c.execute('SELECT * FROM male_likes order by email limit 5;')
print "QUERY:\n", c.fetchall()
c.close()


'\nGiven a DB of entities and their associated likes in JSON, produce a SQL DB of it.\nIf john@google.com has the following "like": {"name":"The Tonight Show Starring Jimmy Fallon",\n"category":"TV Show","id":"31732483895","created_time":"2016-03-26T23:07:58+0000"},\nthen a row in the database would exist with the following:\n\nemail,category,name,id,created_time\njohn@google.com,"TV Show","The Tonight Show Starring Jimmy Fallon","2016-03-26T23:07:58"\n\nThe database is created in males_likes.sqlite\n\nRef. API: https://docs.python.org/2/library/sqlite3.html\n'

Columns:  [u'entity_id', u'_id', u'user', u'customer_id', u'friends', u'likes'] 
 Likes columns:  [u'category', u'created_time', u'name', u'category_list', u'id']


,email,category,name,id,created_time
0,kev312@hotmail.com,Non-Profit Organization,Epic Foundation,450754481767298,2016-04-10T00:34:51+0000
1,kev312@hotmail.com,Non-Profit Organization,French American Chamber of Commerce San Francisco,20624748967,2016-04-09T07:39:15+0000
2,kev312@hotmail.com,Community,Francophones à San Francisco,205317749540643,2016-04-09T07:39:10+0000
3,kev312@hotmail.com,Product/Service,Postmates,205291849553826,2016-04-08T17:01:23+0000
4,kev312@hotmail.com,Local Business,Le Forum des Halles,401822732951,2016-04-08T03:11:38+0000


QUERY:
[(u'08tws@williams.edu', u'Food/Beverages', u'Nova Jersey Exportadora de M\xe9is e C\xearas Ltda', u'889338587877359', u'2016-09-28T22:12:06+0000'), (u'08tws@williams.edu', u'Sports & Recreation', u'ZogSportsDEN', u'1657498024513999', u'2016-09-13T02:15:21+0000'), (u'08tws@williams.edu', u'School Sports Team', u'Williams College Athletics', u'296415946768', u'2016-07-07T20:05:45+0000'), (u'08tws@williams.edu', u'Community', u'Gociety', u'474974892583741', u'2016-07-06T13:11:41+0000'), (u'08tws@williams.edu', u'Health/Wellness Website', u'JaguarFit', u'1714450068796152', u'2016-06-10T15:18:41+0000')]


JSON COMMON ERRORS/ OBSERVATIONS/ TIPS/ Lessons Learnt: 
    
1. Using simplejson.load gives more info on the error: JSONDecodeError: Extra data: line 2 column 1 - line 14486 column 1 (char 15562 - 185136642)

2. json.loads- JSONDecodeError: Expecting value: line 1 column 1 (char 0) error is due to:
            
-non-JSON conforming quoting

-XML/HTML output (that is, a string starting with <), or

-incompatible character encoding

Ultimately the error tells you that at the very first position the string already doesn't conform to JSON.

As such, if parsing fails despite having a data-body that looks JSON like at first glance, try replacing the quotes of the data-body:

```
try: #try parsing to dict
    dataform = str(response_json).strip("'<>() ").replace('\'', '\"')
    struct = json.loads(dataform)
except:
    print repr(resonse_json)
    print sys.exc_info()
```

Note: Quotes within the data must be properly escaped

3. When a json element is malformed:  ValueError: No JSON object could be decoded
        
MAKE SURE:
    
- Test if the encoding of your file is the expected one, in cmd, do:
```
$file -I facebook_males.json 
    facebook_males.json: text/plain; charset=utf-8
```

- The file is closed after open with either with open() as filehandle: or file.close()
    
- Save each entry in an array to overcome the fact that the end of each line is converted to a '\n' that scapes lines. (By default 
json.dumps changes newlines in text content to "\n", keeping your json to a single line)

```
records = [json.loads(line) for line in open(json_path)]
or
entries = []
for line in open(json_path, 'r'):
    entries.append(simplejson.loads(line))
```

- Handle exceptions:
    
```
try:
    j1 = json.loads(r_positive.text)
except ValueError:
    # decoding failed
    continue
else:
    do_next()
```


Other useful refs:
    
-PySQLite: https://charlesleifer.com/blog/using-the-sqlite-json-extension-with-python-old-version-/
        
-http://stackoverflow.com/questions/8811783/convert-json-to-sqlite-in-python-how-to-map-json-keys-to-database-columns-prop    

